# Questions : 
Do we need a historical dataframe for pnl calculations ?


In [9]:
import datetime
import pandas as pd
from modules._instrument import Cash, get_option_from_instrument_id, get_options_from_id_list
from modules._trade import Trade
from modules._historical_data import HistoricalData
# from modules import Options, Instrument, Cash
from modules.global_variables import params

# TODO: to be moved to yaml file
CASH_ID = params['CASH_ID']

# Class POrtfolio

In [ ]:
class Portfolio:

    # Constructor
    # Add one parameter as init_cash -> initial cash
    # amount of cash = init_cash
    # position
    def __init__(self,
                 mkt_data:HistoricalData,
                 portfolio_id = 1, # TODO: it should be string?
                 currency:str = 'INR',
                 owner:str = None,
                 description:str = None,
                 limits:dict = dict(),
                 initial_cash = 0,
                 init_from_file: bool = False,
                 file_name: str = "",
                 time_stamp = None
                 ) -> None:

        self._currency = currency 
        self._owner = owner
        self._description = description
        self._limits = limits # TODO: Will populate this later.
        self._latest_timestamp = time_stamp #_trade.getTime()
        self._portfolio_id = portfolio_id
        self._mkt_data = mkt_data

        self._is_init_cash_added = False

        # call the update method in 

        if init_from_file:
            # TODO :  Code to read the existing portfolio_df
            pass

        else:
            # initializing the portfolio_df
            self._portfolio_df = pd.DataFrame(
                columns=[
                            "instrument_id",
                            "position",
                            "current_price",
                            "weighted_avg_price",
                            "value",
                            "delta", # TODO : to be removed later
                            "trade_id_list",
                            # "mark_to_market"
                            # "realized_pnl",
                        ]
            )

            # Initialise first row of the the portfolio df with 'Cash' instrument with value 'initial_cash' -> call the update method, create a trade list
            self.cash = Cash()
            cash_trade_list = list()
            cash_trade_list.append(Trade(instr_id=self.cash.getId(),
                                    trade_price=self.cash.get_quote(time_stamp,'mid',self._mkt_data)[0],
                                    pos=initial_cash,
                                    trade_time=time_stamp))
            self.update(trade_time=time_stamp, trade_list=cash_trade_list)


    def update_latest_timestamp(self,t)->None:
        '''
        This function will update the latest timestamp in the trade.
        '''
        self._latest_timestamp = t


    def is_instrument_id_in_portfolio(self, instrument_id) -> bool:
        ''' 
        This function will check if an instrument id exists in the portfolio df
        '''
        # check if an instrument_id exists in portfolio df
        ret_val = False
        if str(instrument_id) in self._portfolio_df['instrument_id'].to_list():
            ret_val = True

        return ret_val

    def get_portfolio_option_list(self, )->list:
        '''
        This function gets the list of options from list of ids.

        Returns:
        List of Option objects
        '''
        id_list = self._portfolio_df.loc[self._portfolio_df['instrument_id'] != CASH_ID,['instrument_id']].values.flatten().tolist()
        return get_options_from_id_list(id_list)

    def get_max_expiry_among_instruments(self,)->datetime:
        id_list = self._portfolio_df.loc[self._portfolio_df['instrument_id'] != CASH_ID,['instrument_id']].values.flatten().tolist()
        return self._mkt_data.get_max_expiry_from_options(id_list=id_list)


    # Method to get the current price for an instrument
    def get_current_price(self, 
                          instrument_id, 
                          dt:datetime, 
                          q_type='Mid') -> float:
        ''' 
        This funciton gets the current price with given exchange/instrument id and date_time.
        need to pass the correct q_type = 'bid or ask or mid'. DDG and MM to discuss and decide
        '''
        try: 
            opt = get_option_from_instrument_id(instrument_id,self._mkt_data)
            # print(f'*************{opt}*******************')
            o_price, o_qty = opt.get_quote(dt, q_type, self._mkt_data)
            # print(f'o_price = {o_price}')
            return o_price

        except Exception as e:
            print(e) 


    # Method to get the delta for a given instrument_id
    def get_instrument_delta(self,
                             instrument_id,
                             position,
                             time)-> float:
        ''' 
        This function will get the delta for a specific instrument_id.
        parameters : instrument_id, trade object
        '''
        try:
            type = 'ask' if position < 0 else 'bid'
            delta = get_option_from_instrument_id(instrument_id,mkt_data=self._mkt_data).calculate_delta(t = time, q_type = type, mkt_data=self._mkt_data)
            
        except Exception as e:
            raise e

        return delta


    # method to calculate the weighted average price for a given instrument
    def calculate_weighted_average_price(self, instrument_id, current_last_traded_price,current_position):

        #TODO: think about re-writing this by unpacking the values
        '''
        This function will calculate the weighted average price for a given instrumnet.
        The inputs to the function are instrument_id , current_last_traded_price, current_position.
        '''
        """
        ALGO : 

        - The formula for calculating the weighted average price is : 
          (previous_weghted_average_price x previous quantity + current_last_traded_price x current_quantity)/(previous_quantity + current_quantity)
        - Fetch the previous weighted average price for the given instrument id from the portfolio_df
        - Fetch the previous quantity for the given instrument id from the portfolio_df
        - Fetch the current last traded price for the given instrument id from the trade object
        - Fetch the current quantity for the given instrument id from the trade object
        - supply all these in the formula and calculate the weighted average price
        """

        wap = (
                ((self._portfolio_df.loc[f"{instrument_id}"]["weighted_avg_price"] * 
                self._portfolio_df.loc[f"{instrument_id}"]["position"])+ 
                (current_last_traded_price * 
                current_position)) /
                ((self._portfolio_df.loc[f"{instrument_id}"]["position"]) + 
                (current_position))
            )
        
        return wap

    
    def update(self, trade_time, trade_list:list=list()) -> None:
        """
        This function updates the portfolio_df for each trade.
        """
        # Algo : 
        # - This function will receive a trade_list from the blotter, where trade_list 
        #   is a list of trade objects(trade) and will be updated in portfolio_df.
        # - portfolio_df contains : 'instrument_id','trade_list','portfolio_id','position','last_traded_price',
        #   'current_price','weighted_avg_price','mark_to_market','realized_pnl'
        # - each trade object contains : 'instrument_id','trade_list','portfolio_id','position','last_traded_price',
        # - If a trade happens or trade is populated:
        #       'instrument_id','trade_list','portfolio_id','position','last_traded_price','current_price',
        #       are stored in the portfolio_df
        #       'weighted_avg_price' is calculated and stored in portfolio df
        #       'mark to market' is calculated and stored in portfolio_df.
        #       'realized pnl' is left blank unitl trade stopped.
        #       realized pnl is calculated for the respective instrument,when the position of the instrument is 0.
        # - elif when trade is empty :
        #       update the current price and aslo calculate the pnl.

        try:
            # print("In Update",'*'*30)
            # Checking if trade list is populated with trade
            
            if len(trade_list) != 0 :
                # print("hello")
                # if initial cash is not added, this call is being from the constructor
                # TODO: check whether the fo
                # if not self.is_instrument_id_in_portfolio(instrument_id=CASH_ID):
                if not self._is_init_cash_added:
                    # print("hello 2")
                    # TODO: need to check whether the currency is same
                    trade = trade_list[0]
                    # print(trade)
                    cash_trade_dict = {"instrument_id":[str(trade.getInstrumentId())],
                                       "position":[trade.getPosition()],
                                       "current_price":[trade.getPrice()],
                                       "weighted_avg_price":[trade.getPrice()],
                                       "value":[trade.getPosition()*trade.getPrice()],
                                       "delta" : [self.get_instrument_delta(instrument_id = trade.getInstrumentId(),position= trade.getPosition(), time = trade.getTime())], # to be removed later , initializing the delta
                                       "trade_id_list":['0'], # dummy trade id
                                       # "mark_to_market":,
                                       # # "realized_pnl"}
                                      }

                    # adding initial cash in the portfolio_df
                    # self._portfolio_df = self._portfolio_df.append(pd.DataFrame(cash_trade_dict, index=[0]))
                    self._portfolio_df = self._portfolio_df.append(pd.DataFrame(cash_trade_dict), ignore_index=True)
                    print(f'{trade.getPosition()} {self.getCurrency()} cash added to portfolio.')
                    # print(self._portfolio_df)
                    self._is_init_cash_added = True
                else:
                    # iterating over the trade_list object
                    # print("In Update",'#'*30)
                    # accumulate the cash in the current trade list
                    accumulated_cash = 0
                    for trade in trade_list:
                        # print(self._portfolio_df) 
                        quote_type = 'ask' if trade.getPosition() < 0 else 'bid'
                        curr_price = self.get_current_price(trade.getInstrumentId(),trade_time,quote_type)
                        # unpacking trade objects in a trade_dict and assigning the default values
                        # print("Im here")

                        trade_dict = {
                                        "instrument_id": [str(trade.getInstrumentId())],
                                        # "portfolio_id": trade.getPortfolioId(),
                                        # "last_traded_price": trade.getPrice(), TODO: do we need this?
                                        "position": [trade.getPosition()],
                                        'current_price' : [curr_price],
                                        "weighted_avg_price" : [trade.getPrice()],
                                        "value":[trade.getPosition()*trade.getPrice()],
                                        "delta":[self.cash.calculate_delta(t = trade.getTime(),q_type ='ask' if trade.getPosition() < 0 else 'bid',mkt_data=self._mkt_data )],
                                        "trade_id_list": [trade.getTradeId()]
                                        # "mark_to_market" : 0, 
                                        # "realized_pnl" : 0,
                                    }
                        accumulated_cash += (trade.getPosition() * trade.getPrice())
                        # print(f'Trade Dict : {trade_dict}')
                        # print(self._portfolio_df.instrument_id.to_list(),"   -    ",  trade_dict['instrument_id'][0])
                        # Checking whether if the new instrument id is present in the portfolio_df
                        # TODO: use inst
                        # self.instrument_id_in_portfolio()
                        # if self.is_instrument_id_in_portfolio(instrument_id=trade_dict["instrument_id"][0]):
                        if (trade_dict["instrument_id"][0] in self._portfolio_df.instrument_id.to_list()):
                            # print("In if statement", "$"*20)
                            self._portfolio_df.set_index("instrument_id", inplace=True)
                            # appending the trade_list for the corresponding instrument_id
                            # portfolio_id is constant for all trades
                            # self._portfolio_df.loc[f"{trade_dict['instrument_id']}"]["portfolio_id"] = trade_dict["portfolio_id"]
                            # updating the traded price for the corresponding instrument id
                            # self._portfolio_df.loc[f"{trade_dict['instrument_id']}"]["last_traded_price"] = trade_dict["last_traded_price"]
                            # adding the current price
                            self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["position"] = (
                                                                                                    self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["position"] + 
                                                                                                    trade_dict["position"][0]
                                                                                                    )
                            self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["current_price"] = trade_dict['current_price'][0]
                            # adding the weighted average price  for the corresponding instrument id : (previous_wt_avg_price x prev_qty + today_price x today_qty)/total_qty
                            # print(f'''\nFor {trade_dict["instrument_id"][0]} : Weighted average price {self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["weighted_avg_price"]} Position : {self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["position"]}''')
                            # self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["weighted_avg_price"] = (
                            #                                                                                     ((self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["weighted_avg_price"] * 
                            #                                                                                     self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["position"])+ 
                            #                                                                                     (trade.getPrice() * 
                            #                                                                                     trade_dict["position"][0])) /
                            #                                                                                     (self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["position"] + 
                            #                                                                                     trade_dict["position"][0])
                            #                                                                                     )
                            self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["weighted_avg_price"] = self.calculate_weighted_average_price(instrument_id= trade_dict['instrument_id'][0],
                                                                                                                                                        current_last_traded_price= trade.getPrice(),
                                                                                                                                                        current_position = trade_dict["position"][0])
                            # print(f'''\nFor {trade_dict["instrument_id"][0]} : Updated Weighted average price {self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["weighted_avg_price"]}''')
                            # adding the positions for the corresponding instrument id

                            self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["value"] = (
                                                                                                    self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["value"] +
                                                                                                    trade.getPrice() * trade_dict["position"][0]
                                                                                                    )
                            self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["trade_id_list"] = (
                                                                                                        str(self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["trade_id_list"]) + 
                                                                                                        "," + 
                                                                                                        str(trade_dict["trade_id_list"][0])
                                                                                                        )

                            self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["delta"]=self.get_instrument_delta(instrument_id = trade_dict['instrument_id'][0],position= trade_dict["position"][0], time = trade.getTime())
                            # print(f"\n\n DELTA : {self.get_instrument_delta(instrument_id = trade_dict['instrument_id'][0],position= trade_dict['position'][0], time = trade.getTime())}\n\n")
                            # adding the mark to market for the corresponding instrument id : Mark to market = position(current price - weighted average price)
                            # print(f'''port position : {self._portfolio_df.loc[f"{trade_dict['instrument_id'][0]}"]["position"]}''')
                            # print(f'Position : {trade_dict["position"][0]}')
                            # self._portfolio_df.loc[f"{trade_dict['instrument_id']}"]["mark_to_market"] = (
                            #                                                                             (self._portfolio_df.loc[f"{trade_dict['instrument_id']}"]["position"]) * (trade_dict['current_price'] - 
                            #                                                                             self._portfolio_df.loc[f"{trade_dict['instrument_id']}"]["weighted_avg_price"])
                            #                                                                             )
                            ### doubt in part below # When will I get the position = 0, from unwind ?
                            # adding the realized pnl for the corresponding instrument id
                            # TODO : Code to update the realized pnl value

                            self._portfolio_df.reset_index(inplace=True)

                        else:
                            # print("In else statement", "^"*20)
                            # adding the trade details for the instrument id not prensent in the portfolio_df
                            # self._portfolio_df = self._portfolio_df.append(pd.DataFrame(trade_dict, index=[0]))
                            self._portfolio_df = self._portfolio_df.append(pd.DataFrame(trade_dict), ignore_index=True)
                            
                            # TODO : Update cash
                            # self._portfolio_d[f

                            # update cash when trades executed
                            self.update_portfolio_cash(-1 * accumulated_cash)
                            # Logic =  Cash generated is position x price x -1, cash is only generated when a new trade is generated or when position is closed.
                            # TODO : update cash check during unwind.

                    if params['DEBUG']:
                        print(f'\n')
                        print('-'*80)
                        print(f"Total portfolio value at time t {trade.getTime()} is : {self._portfolio_df['value'].sum()}")
                        print(f'\n')
                        print('-'*80)
                        print('Portfolio df after each trade')
                        print(f'\n')
                        print(self._portfolio_df)
                        print(f'\n')
                        print('-'*80)            
                        # TODO: updating backtest at each time step
                        # self.back_test.update()
            
            # update current prices and corresponding values at each time step
            self.update_current_prices_values(trade_time)



        except Exception as e:
            raise e


    def update_portfolio_cash(self, accumulated_cash):
        '''
        updates the cash position and value in the portfolio
        '''
        # its assumed that there will be only one row for cash with instrumentid as CASH_ID
        self._portfolio_df.loc[self._portfolio_df['instrument_id']==CASH_ID,['position','value']] = \
        self._portfolio_df.loc[self._portfolio_df['instrument_id']==CASH_ID,['position','value']] + accumulated_cash 


    def update_current_prices_values(self,utime):
        '''
        Updates the current prices and corresponding values of the instruments in the portfolio
        '''
        try:
            # update the current price for each instrument id in the portfolio_df
            self._portfolio_df.set_index("instrument_id", inplace=True)
            # print('#'*100)
            # print(self._portfolio_df)
            cash_val_change = 0
            for id in self._portfolio_df.index:
                if id == CASH_ID:
                    # TODO: need to change CASH_ID
                    continue

                # TODO: the following updated ahould be clubbed together into one update
                # updating the current price for the respective instrument_id
                # print(f"portfolio_df.index={id}")
                curr_price = self.get_current_price(id,utime,'mid')
                # print(curr_price)
                self._portfolio_df.loc[id]['current_price'] = curr_price
                # updating the mark to market price for the respective instrument_id
                # TODO: we can update MTM later
                # self._portfolio_df.loc[id]["mark_to_market"] = (self._portfolio_df.loc[id]['position'] *
                #                                                 (self._portfolio_df.loc[id]['current_price'] - 
                #                                                 self._portfolio_df.loc[id]["weighted_avg_price"])
                #                                                 ) 
                self._portfolio_df.loc[id]['value'] = curr_price * self._portfolio_df.loc[id]['position']
                # cash_val_change += 
            self._portfolio_df.reset_index(inplace=True)

        except Exception as e:
            raise e


    def get_portfolio_delta(self, qtime):
        '''
        This function returns the portfolio delta at a particular given time

        Parameters:
        qtime : time 

        pseudo code:
            get the instrumnet object by id if not cash
            from the instrument object call calculate_delta()
        '''
        # TODO: current assumption is that, this portfolio will contain only options

        # TODO: Q: Do we need to send time as well while getting portfolio_delta? 
        #          i.e we should get_delta at a particular time?
        #          OR
        #          or anytime this method is called?

        def compute_delta(row):
            row_delta = (get_option_from_instrument_id(row,self._mkt_data)).calculate_delta(qtime,'mid',self._mkt_data)
            # print(f'found option with id={row}')
            return row_delta

        delta_sum = self._portfolio_df['instrument_id'].apply(compute_delta).sum()
        
        return delta_sum

    def get_instrument_total_pnl(self, instrument_id)-> float:
        '''
        This function will give the total pnl for a given instrument_id.
        '''
        """
        ALGO : 

        - For a given instrument_id this func will locate the instrument id in the portfolio_df
        - Get the RPNL
        - Get the MTM
        - total_pnl = RPNL + MTM
        """

        self._portfolio_df.set_index('instrument_id', inplace = True)
        # Fetching the mark to market for the specific instrument id from the portfolio_df
        mtm = self._portfolio_df.loc[instrument_id]['mark_to_market']
        # Fetching the realized pnl for the specific instrument id from the portfolio_df
        rpnl = self._portfolio_df.loc[instrument_id]['realized_pnl']
        self._portfolio_df.reset_index(inplace = True)
        # Adding the mark to market and the realized pnl for the specific instrument_id
        total_pnl = mtm + rpnl

        return (total_pnl,mtm,rpnl)


    def get_portfolio_total_pnl(self)-> float:
        '''
        This function will give the total pnl of the portfolio.
        '''
        """
        ALGO : 

        - Get the sum of the mark to market row in the portfolio_df and store it in MTM
        - Get the sum of the realized pnl in the portfolio_df and store it in RPNL
        - total_pnl = RPNL + MTM
        """
        # taking the sum of all the instrument's mark to market in the portfolio_df
        mtm = self._portfolio_df['mark_to_market'].sum()
        # taking the sum of all the instrument's realized pnl in the portfolio_df
        rpnl = self._portfolio_df['realized_pnl'].sum()

        # sum the mtm and the realized pnl
        total_pnl = mtm + rpnl

        return (total_pnl,mtm,rpnl)


    def get_portfolio_value(self, update=False, utime=0):
        '''
        This method will return the present portfolio value
        '''
        # TODO: if update :
        #           update the portfolio at the utime
        #       return the portfolio value
        if update:
            self.update_current_prices_values(utime=utime)

        portfolio_val = self._portfolio_df['value'].sum()

        return portfolio_val


    def view(self) -> None:
        """
        This function is used to view the portfolio_df at the end of each trade cycle.
        This function is only Test specific (Debugging)
        """
        try:
            print("\n", "*" * 30)
            print("portfolio state")
            print("-" * len("portfolio state"))
            print(self._portfolio_df)
            print("*" * 30)

        except Exception as e:
            print(e)


    def serialize(self,):
        # TODO: File name should contain 
        self._portfolio_df.to_csv('')


    # Dunder methods __str__ and __repr__

    def __str__(self):
        print('ID: {0}  Currency {1} Description {2} Timestamp {3} \n'.format(self.getPortfolio_id(), self.getCurrency(), self.getDescription(), self.get_latest_timestamp() ))
        print(self.getPortfolio_df())

    def __repr__(self):
        print(f' ID : {self.getPortfolio_id()}\
                Currency : {self.getCurrency()}\
                Description : {self.getDescription()}\
                Timestamp : {self.get_latest_timestamp()}')

        print(f'Portfolio_df : {self.getPortfolio_df()}')

    # All Getter methods

    def getCash(self):
        # TODO the first row of the df
        pass
    

    def getCurrency(self)->str:
        '''
        This function will get the currency from the constructor.
        '''
        return self._currency

    def getOwner(self)->str:
        '''
        This function will get the owner from the constructor.
        '''
        return self._owner

    def getDescription(self)->str:
        '''
        This function will get the description from the constructor.
        '''
        return self._description

    def getLimits(self)->dict:
        '''
        This function will get the limits from the constructor.
        '''
        return self._limits
    
    def getPortfolio_id(self)->int:
        '''
        This function will get the limits from the constructor.
        '''
        return self._portfolio_id
    
    def get_latest_timestamp(self):
        '''
        This function will get the limits from the constructor.
        '''
        return self._latest_timestamp
    
    def getPortfolio_df(self):
        return self._portfolio_df
    



# All setter methods

    def setCash(self, amount):
        # TODO: 
        pass

    def setCurrency(self,currency)->None:
        '''
        This function will set the currency.
        '''
        self._currency = currency

    def setOwner(self,owner)->None:
        '''
        This function will set the owner.
        '''
        self._owner = owner    

    def setDescription(self,description)->None:
        '''
        This function will set the description.
        '''
        self._description = description

    def setLimits(self,limits)->None:
        '''
        This function will set the limits.
        '''
        self._limits = limits


In [11]:
import warnings
warnings.filterwarnings("ignore")

from modules._historical_data import HistoricalData
from modules._instrument import get_option_from_instrument_id

hist_data = HistoricalData(source="eis_data", name="EIS", underlying_instrument="BANKNIFTY", start_date="20210310", end_date="20210310")
hist_data.load_market_data()

get_option_from_instrument_id("35023", hist_data).calculate_delta("2021-03-10 09:16:00", "bid", hist_data)

0.6279618069129356

In [ ]:
delta = get_option_from_instrument_id(35023,mkt_data=self._mkt_data).calculate_delta(t = time, q_type = type, mkt_data=self._mkt_data)

In [1]:
import os
from modules.global_variables import params

In [2]:
os.makedirs(params['OBJ_STORE'], exist_ok=True)